In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
from itertools import product
from rich.progress import track

In [2]:
rng = np.random.default_rng(0)

In [4]:
# verify hdf5 file
p_h5 = "../data/tables/20240621/clamp_embeddings.h5"
with h5py.File(p_h5, "r") as hdf:
    print(f"{len(hdf.keys())} rows in file")
    df = pd.DataFrame(index=list(hdf.keys()), columns=["embeddings"])
    # Iterate over all items in the HDF5 file
    nk = 0
    for key in track(hdf.keys()):
        df.loc[key, "embeddings"] = hdf[key][:]  # type: ignore
df.head()
df.to_hdf("../data/tables/20240621/clamp_embeddings_new.h5", key="embeddings")

Output()

545952 rows in file


/var/folders/8c/sxhlb80n0c93ft54s3hhn5yw0000gn/T/ipykernel_15858/1817444113.py:11: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['embeddings'], dtype='object')]

  df.to_hdf("../data/tables/20240621/clamp_embeddings_new.h5", key="embeddings")


In [ ]:
# load prompt dataset
p_test = "../data/datasets/test/dataset samples"
transformations = [f"t{t:02d}s{s:02d}" for t, s in product(range(12), range(8))]
q_files = [f"{f[:-4]}_{rng.choice(transformations)}" for f in os.listdir(p_test)]
q_files[:3]

In [ ]:
# load prompt embeddings
q_embeddings = [embedding_dataset[key] for key in q_files]
q_embeddings[0]

In [ ]:
embedding_dataset.values()

In [ ]:
# find best and worst n matches for prompts
n_matches = 3
indices = []  # To store the indices of the best and worst matches
for q_emb in q_embeddings:
    similarities = []
    for idx, dataset in enumerate(embedding_dataset.values()):
        sim = 1 - cosine(q_emb, dataset)
        similarities.append((sim, idx))

    similarities.sort(key=lambda x: x[0], reverse=True)  # Highest first
    best_indices = [idx for _, idx in similarities[:n_matches]]  # Top 3
    worst_indices = [idx for _, idx in similarities[-n_matches:]]  # Bottom 3

    indices.append((best_indices, worst_indices))

In [ ]:
embedding_dataset.close()